# Theoretical Foundations of Computer Science II COMP2870

**School of Computer Science, University of Leeds**

# Lab03: Linear systems of equations

These labsheets contains *formative activities* (which contribute to your learning) and *summative activities* (which you will complete and submit to be assessed as part of your portfolio).

<div class="alert alert-block alert-danger">
<b>Portfolio exercise</b>

Exercise marked with a red box is a summative exercise and must be submitted as part of your portfolio. You should use Gradescope to submit portfolio activities.
</div>

### Expectations

1. **Timeliness** You should complete all of the activities in the order provided and submit your portfolio evidence on Gradescope before the completion date (Friday 14 November, 5pm).

2. **Presentation** You should present all of your work clearly and concisely following additional guidance below.

3. **Integrity** You are responsible for ensuring that all the evidence you submit as part of your portfolio is entirely your own work. You can find out more about Academic integrity on the Skill@library website. All work you submit for assessment is subject to the academic integrity policy.

### Feedback
Feedback on formative activities will be provided via lab classess and tutorials. Feedback on evidence submitted as part of the portfolio will be available on Gradescope.

### Support opportunities
Support with the activity sheet is available in the lab classes and tutorials. Individual support is available via the online booking system.

### Statement on the Use of Generative AI (Red Category)
This assessment is RED according to the GenAI traffice light system. **Generative AI (GenAI) tools cannot be used**. The aim of this task is for you to develop and demonstrate the specific skills and knowledge covered in the taught sessions. We want you to independently develop your understanding, criticical thinking skills and demonstrate fundamental skills that will be required throughout your programme. Reliance on GenAI could prevent you from achieving the intended learning outcomes and may impeded your skill development.

You are still permitted to use dictionaries, thesauri, spelling and grammer-checking software to help identify and correct spelling mistakes and grammatrical errors (even if they are powered by GenAI). However, you should not use any software to rewrite sentence or make substantial changes to your original text, as this would be against the rules of this category.

Failure to comply with these requirements may be considered academic misconduct under University regulations.

### Expected time for completion:
1-2 hours

### Expected completion date:
Friday 14 November, 5pm

## Coursework summary

This lab covers material on how to solve systems of linear equations.

## Learning outcomes

On completion of this activity sheet you will have demonstrated that you can:

- apply direct and iterative solvers to solve systems of linear equations; implement methods using floating point numbers and investigate computational cost using computer experiments

## How to access the lab

You can access the lab worksheet directly through [minerva](https://minerva.leeds.ac.uk/), [github](https://github.com/COMP2870-2526/linear-algebra-student-labs) or using Noteable (accessible via Minerva - see `README.md` for detailed instructions).

These lab worksheets are written using ['Jupyter Notebooks'](https://jupyter.org/). Many, many tutorials and guides are [available online](https://www.dataquest.io/blog/jupyter-notebook-tutorial/).

First some helper code for generating good matrices for LU factorisation tests. 

In [1]:
import numpy as np
from scipy.sparse import diags


def generate_safe_system(n):
    """
    Generate a linear system A x = b where A is strictly diagonally dominant,
    ensuring LU factorization without pivoting will work.

    Parameters:
        n (int): Size of the system (n x n)

    Returns:
        A (ndarray): n x n strictly diagonally dominant matrix
        b (ndarray): RHS vector
        x_true (ndarray): The true solution vector
    """

    k = [np.ones(n - 1), -2 * np.ones(n), np.ones(n - 1)]
    offset = [-1, 0, 1]
    A = diags(k, offset).toarray()

    # Solution is always all ones
    x_true = np.ones((n, 1))

    # Compute b = A @ x_true
    b = A @ x_true

    return A, b, x_true

## Exercise 1: LU factorisation

<div class="alert alert-block alert-danger">
<b>Portfolio exercise</b>
You should submit:

1. Your code for `lu_factorisation`
2. The determinant of `A_large` from Exercise 1.3.
3. The plot of run times for LU factorisation (your code) and Gaussian elimination (from the notes)
</div>

1. Implement the algorithm for LU factorisation as described [in the
notes](https://comp2870-2526.github.io/linear-algebra-notes/src/lec04.html#sec-LU-factorisation).
You should implement a Python function which accepts an $n \times n$
matrix $A$ represented as a numpy array and returns one lower triangular
matrix $L$ and one upper triangular matrix $U$ with $A = LU$.

    Here is some starter code:

In [2]:
def lu_factorisation(A):
    """
    Compute the LU factorisation of a square matrix A.

    The function decomposes a square matrix ``A`` into the product of a lower
    triangular matrix ``L`` and an upper triangular matrix ``U`` such that:

    .. math::
        A = L U

    where ``L`` has unit diagonal elements and ``U`` is upper triangular.

    Parameters
    ----------
    A : numpy.ndarray
        A 2D NumPy array of shape ``(n, n)`` representing the square matrix to
        factorise.

    Returns
    -------
    L : numpy.ndarray
        A lower triangular matrix with shape ``(n, n)`` and unit diagonal.
    U : numpy.ndarray
        An upper triangular matrix with shape ``(n, n)``.
    """
    n, m = A.shape
    if n != m:
        raise ValueError(f"Matrix A is not square {A.shape=}")

     # construct arrays of zeros
    L, U = np.zeros_like(A), np.zeros_like(A)

    # ...
    for i in range (n):
        for j in range (i, n):
            U[i, j] =  A[i, j] - np.sum(L[i, :i] * U[:i, j])
    
    # set the diagonal of L to be 1
    L[i, i] = 1

    for j in range(i + 1, n):
        if U[i, i] == 0:
            raise ZeroDivisionError("Zero pivot encountered- consider using pivoting.")
        L[j, i] = (A[j, i] - np.sum(L[j, :i] * U[:i, i])) / U[i, i]

    return L, U

2. Test your solution on the matrix $A$:
   $$
   A = \begin{pmatrix}
   4 & 2 & 0 \\ 2 & 3 & 1 \\ 0 & 1 & 2.5
   \end{pmatrix}.
   $$
   First compute the LU factorisation by hand and then check $A = L U$.

In [3]:
import numpy as np
A = np.array([
    [4, 2, 0],
    [2, 3, 1],
    [0, 1, 2.5]
])

L, U = lu_factorisation(A)

print("L = \n", L)
print("\nU = \n", U)
print("\nCheck A = L@U", np.allclose(A, L @ U))

L = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

U = 
 [[4.  2.  0. ]
 [0.  3.  1. ]
 [0.  0.  2.5]]

Check A = L@U False


3. Consider the large matrix given by:

In [4]:
A_large, b_large, x_large = generate_safe_system(100)

We can use the LU factorisation as another way to compute the determinant of a matrix.
We can use the fact that we can easily compute the determinant of triangular matrix and that $\det(XY) = \det(X) \det(Y)$.

Use this following code to determine the determinant of `A`:

In [5]:
def determinant(A):
    n = A.shape[0]
    L, U = lu_factorisation(A)

    det_L = 1.0
    det_U = 1.0

    for i in range(n):
        det_L *= L[i, i]
        det_U *= U[i, i]

    return det_L * det_U


4. Combine your implementation of LU factorisation with the forwards and
backwards solve from the lecture notes to create a solver for a system
of linear equations. Compare how long it takes for each part of your
code to run against the code for Gaussian elimination from the lecture
notes.

    Use the following code as a starter:

In [ ]:

def gaussian_elimination(A, b):
    """Convert A to upper-triangular (in place) using forward elimination."""
    n = len(b)
    for i in range(n - 1):
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            A[j] -= factor * A[i]
            b[j] -= factor * b[i]

def backward_substitution(U, y):
    """Solve Ux = y for upper-triangular U."""
    n = len(y)
    x = np.zeros_like(y, dtype=float)

    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

    return x

def solve_via_ge(A, b):
    """Solve Ax = b using Gaussian Elimination."""
    A = A.astype(float).copy()
    b = b.astype(float).copy()

    gaussian_elimination(A, b)
    return backward_substitution(A, b)

def forward_substitution(L, b):
    """Solve Ly = b where L has 1s on the diagonal."""
    n = len(b)
    y = np.zeros_like(b, dtype=float)

    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])

    return y

def solve_via_lu(A, b):
    """Solve Ax = b using LU factorization."""
    L, U = lu_factorisation(A)

    y = forward_substitution(L, b.astype(float))
    return backward_substitution(U, y)


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.sparse import diags


sizes = [2**j for j in range(1, 6)]

for n in sizes:
    # generate a random system of linear equations of size n
    A, b, x = generate_safe_system(n)

    # do the solve
    ...
    start_time = time.time()
    L, U = lu_factorisation(A)
    end_time = time.time()
    time_gone = end_time - start_time

    print(f"n={n:4d}, time taken for LU factorisation: {time_gone:.6f} seconds")
    print(f"Determinant of A: {determinant(A)}")

for n in sizes:
    A, b, x_true = generate_safe_system(n)
    start = time.time()
    x = solve_via_ge(A, b)
    end = time.time()
    elapsed = end - start
    times_ge.append(elapsed)
    print(f"n={n:>3}  took {elapsed:.6f} s")


n=   2, time taken for LU factorisation: 0.000039 seconds
Determinant of A: 0.0
n=   4, time taken for LU factorisation: 0.000053 seconds
Determinant of A: 0.0
n=   8, time taken for LU factorisation: 0.000148 seconds
Determinant of A: 0.0
n=  16, time taken for LU factorisation: 0.000510 seconds
Determinant of A: 0.0
n=  32, time taken for LU factorisation: 0.001965 seconds
Determinant of A: 0.0



## Exercise 2: LU factorisation with pivoting (Extension)

We can improve the stability of LU factorisation by additionally
including a pivoting step. In this way, we compute a factorisation:
$$
P A = L U.
$$

The new algorithm looks like:

1.  Initialise $P = I_n$ (the $n \times n$ identity), $L = O_n$ (the
    $n \times n$ zero matrix) and $U = O_n$.

2.  For $k = 1, \ldots n$:

    1.  Find the pivot row with the largest $|A_{ik}|$ for
        $i = k, \ldots, n$. Swap rows $k$ and $p$ in $A$ and $L$ and store this
        swap in $P$.
    2.  Set $L_{kk} = 1$.
    3.  Continue with LU factorisation in column $k$ as usual.
    4.  Repeat for next column $k+1$.

Implement this new method and test on the system of linear equations
given by:
$$
\begin{pmatrix}
10 & -7 & 0 \\ -3 & 2.1 - \varepsilon & 6 \\ 5 & -1 & 5
\end{pmatrix}
\begin{pmatrix}
x_1 \\ x_2 \\x_3
\end{pmatrix}
= \begin{pmatrix}
7 \\ 9.9 + \varepsilon \\ 11
\end{pmatrix}.
$$
Test with $\varepsilon = 10^{-14}$.